### 경사하강을 이용한 행렬 분해

In [2]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재요인 차원 K는 3 설정. 
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN ],
              [np.NaN, 5, np.NaN, 3, 1 ],
              [np.NaN, np.NaN, 3, 4, 4 ],
              [5, 2, 1, 2, np.NaN ]])
num_users, num_items = R.shape
R.shape

(4, 5)

In [3]:
# P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 random한 값으로 입력합니다. 
np.random.seed(1)
K=3
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [4]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    # 두개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)
    
    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
      
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

In [9]:
# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장. 
non_zeros = [(i, k, R[i,k]) for i in range(num_users) for k in range(num_items) if R[i,k] > 0 ]

steps=1000
learning_rate=0.01
r_lambda=0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트. 
for step in range(steps):
    for i, k, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eik = r - np.dot(P[i, :], Q[k, :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eik * Q[k, :] - r_lambda*P[i,:])
        Q[k,:] = Q[k,:] + learning_rate*(eik * P[i, :] - r_lambda*Q[k,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
        print(f"### iteration step: {step:3d}, rmse: {rmse:.6f}")

### iteration step:   0, rmse: 0.016411
### iteration step:  50, rmse: 0.016373
### iteration step: 100, rmse: 0.016334
### iteration step: 150, rmse: 0.016294
### iteration step: 200, rmse: 0.016253
### iteration step: 250, rmse: 0.016211
### iteration step: 300, rmse: 0.016170
### iteration step: 350, rmse: 0.016128
### iteration step: 400, rmse: 0.016087
### iteration step: 450, rmse: 0.016045
### iteration step: 500, rmse: 0.016004
### iteration step: 550, rmse: 0.015964
### iteration step: 600, rmse: 0.015924
### iteration step: 650, rmse: 0.015884
### iteration step: 700, rmse: 0.015845
### iteration step: 750, rmse: 0.015807
### iteration step: 800, rmse: 0.015769
### iteration step: 850, rmse: 0.015732
### iteration step: 900, rmse: 0.015695
### iteration step: 950, rmse: 0.015659


In [8]:
# 예측 행렬
pred_matrix = np.dot(P, Q.T)
np.round(pred_matrix, 3)

array([[3.991, 0.897, 1.306, 2.002, 1.663],
       [6.696, 4.978, 0.979, 2.981, 1.003],
       [6.677, 0.391, 2.987, 3.977, 3.986],
       [4.968, 2.005, 1.006, 2.017, 1.14 ]])